# Medical Imaging Workshop - Notebook 2: Model Training on EyePop.ai
Ingest dataset into EyePop.ai and view

In [ ]:
%pip install fiftyone eyepop tqdm pycocotools

In [ ]:
import os
import asyncio
from tqdm.notebook import tqdm
import asyncio
from importlib import resources
from eyepop import EyePopSdk
from eyepop.data.data_endpoint import DataEndpoint
from eyepop.data.data_jobs import DataJob
from eyepop.data.data_types import DatasetCreate, AssetImport, \
    AutoAnnotateParams, Dataset, Asset, ChangeEvent, ChangeType, DatasetUpdate, UserReview, \
    Model, ModelCreate, ModelStatus
from pycocotools.coco import COCO


In [ ]:
# 🔧 Configuration
accountUUID = "<ENTER YOUR ACCOUNT UUID HERE>"
accountUUID = "2c97ab0b556742dbbdb7af34cc6f3b6a"

apikey = "<ENTER YOUR API KEY HERE>"
apikey = "AAE_w6lCcrCa27chNAbZO-WdZ0FBQUFBQmwyUFk5bmtLZnJBQ2RFVWVDbzU1MnkwTUMzYXhQWjA4a0ZEczFKWWdONjdRS0NGWUZ5aF90aXVQZ3FrcWdkZWwwUEx6Q0luM0F3b3ItMjdqRmhUQkxyTWVvSndFLWRCUENjZGNlanZhbGhRTDdtV289"
datasetUUID = None

#training_directory = "./.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/0688195453827ae280000526e30dc7a9"
training_directory = ".cache/voxel51/Training Set Coco Format"
dataset_name = "Stenosis Dataset" 

In [ ]:
from tqdm.notebook import tqdm
from pathlib import Path
from PIL import Image
from eyepop.data.data_types import AssetImport, Prediction

# Converts COCO annotations to EyePop AssetImports with full ground truth Predictions
def coco_to_eyepop_imports(coco_json_path, image_dir):
    coco = COCO(coco_json_path)
    images = coco.loadImgs(coco.getImgIds())
    annotations = coco.loadAnns(coco.getAnnIds())

    # Map image_id -> file_name
    image_map = {img['id']: img['file_name'] for img in images}

    # Collect annotations per image
    asset_map = {}
    for ann in tqdm(annotations, desc="Processing Annotations"):
        img_id = ann['image_id']
        file_name = image_map[img_id]
        file_path = os.path.join(image_dir, file_name)
        label = coco.loadCats([ann['category_id']])[0]['name']
        bbox = ann['bbox']

        obj = {
            "id": ann['id'],
            "confidence": 1.0,
            "classLabel": label,
            "category": label,
            "traceId": None,
            "x": bbox[0],
            "y": bbox[1],
            "width": bbox[2],
            "height": bbox[3],
            "orientation": 0,
            "outline": None,
            "contours": None,
            "mask": None,
            "objects": None,
            "classes": None,
            "texts": None,
            "meshs": None,
            "keyPoints": None
        }

        if file_path not in asset_map:
            asset_map[file_path] = []

        asset_map[file_path].append(obj)

    # Build AssetImport list
    asset_imports = []
    for file_path, objects in tqdm(asset_map.items(), desc="Building Asset Imports"):
        width, height = get_image_dimensions(file_path)
        prediction = Prediction(
            source_width=width,
            source_height=height,
            objects=objects
        )

        asset_imports.append(AssetImport(
            externalId=os.path.basename(file_path),
            url=file_path,
            ground_truth=prediction
        ))

    return asset_imports

# Utility function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

In [ ]:
async def upload_image(endpoint, dataset_uuid, path):
    with open(path, "rb") as f:
        job = await endpoint.upload_asset_job(f, mime_type="image/jpeg", dataset_uuid=dataset_uuid, external_id=path)
        upload_result = await job.result()

    asset_uuid = getattr(upload_result, 'uuid', None) or getattr(upload_result, 'id', None)  # fallback keys

    if not asset_uuid:
        raise Exception("Upload response missing asset UUID")

    while True:
        asset = await endpoint.get_asset(asset_uuid, dataset_uuid=dataset_uuid)
        status = getattr(asset, 'status', None)
        if status == 'accepted':
            break
        else:
            await asyncio.sleep(.5)
            
    return asset

async def upload_assets_individually(endpoint, dataset_uuid, asset_imports):
    assets = []
    tasks = []   

    for asset_import in tqdm(asset_imports, desc="Scheduling Asset Imports"):
        asset_current = await upload_image(endpoint, dataset_uuid, asset_import.url)        
        await endpoint.update_asset_ground_truth(asset_uuid=asset_current.uuid, dataset_uuid=dataset_uuid, ground_truth=asset_import.ground_truth)
        
    await asyncio.gather(*tasks)
    return assets

In [ ]:
async def run():
    global datasetUUID
    
    async with EyePopSdk.dataEndpoint(secret_key=apikey, account_id=accountUUID, is_async=True, disable_ws=False) as endpoint:
        if(datasetUUID is None):
            dataset = await endpoint.create_dataset(DatasetCreate(name=dataset_name))
            datasetUUID = dataset.uuid
            print("created dataset: %s", datasetUUID)

        annotations_path = os.path.join(training_directory, "annotations", "annotations.json")
        images_path = os.path.join(training_directory, "data")  # adjust if needed

        asset_imports = coco_to_eyepop_imports(annotations_path, images_path)
        print("importing %d assets to dataset %s", len(asset_imports), datasetUUID)
        assets = await upload_assets_individually(endpoint, datasetUUID, asset_imports)
        print(f"✅ Imported {len(assets)} assets.")

await run()

In [ ]:
import webbrowser

webbrowser.open(f"https://dashboard.eyepop.ai/wizardModel?type=object&step=autoLabel&accountUUID={accountUUID}&datasetUUID={datasetUUID}")